In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import os
import torch

import torch.nn as nn
import torch.optim as optim

from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import VGG19

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/vgg19/models//'

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-2000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 50
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0

In [4]:
working_folder = 'notebooks/vgg19'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
vgg = VGG19()

if CONTINUE_TRAINING:
    model = vgg.load(MODEL_WEIGHTS_PATH)
else:
    model = vgg.load()

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
        
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
             
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Adm/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100.0%


In [6]:
criterion = nn.CrossEntropyLoss()

# Only the fully connected layer is being optimized
# optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=LR)
optimizer = optim.Adamax(model.parameters(), lr=LR)

# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()
step_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA)

In [7]:
dataset = PytorchDataset(DATA_DIR, vgg.data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)
model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/50
----------
train Loss: 0.7348 Acc: 0.4926
val Loss: 0.6931 Acc: 0.5070
Epoch duration: 8 m 15s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 2/50
----------
train Loss: 0.6931 Acc: 0.5107
val Loss: 0.6930 Acc: 0.5070
Epoch duration: 15 m 57s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 3/50
----------
train Loss: 0.6932 Acc: 0.5107
val Loss: 0.6931 Acc: 0.5070
Epoch duration: 14 m 49s
Learning Rate = 0.001
Loss didnt decrease from 0.693048078409383. Increasing patience counter

Epoch 4/50
----------
train Loss: 0.6932 Acc: 0.5092
val Loss: 0.6931 Acc: 0.5070
Epoch duration: 14 m 43s
Learning Rate = 0.001
Loss didnt decrease from 0.693048078409383. Increasing patience counter

Epoch 5/50
----------
train Loss: 0.6930 Acc: 0.5107
val Loss: 0.6931 Acc: 0.5070
Epoch duration: 15 m 1s
Learning Rate = 0.001
Loss didnt decrease from 0.693048078409383. Increasing patience counter

Epoch 6/50
----------


KeyboardInterrupt: 